# 8.25习题课
# 1.第七周作业讲解
# 2.项目一问题分别解答

# 新华社新闻抄袭

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import jieba
import scipy
import sklearn
import time, timeit
import re

In [3]:
data = pd.read_csv('sqlResult_1558435.csv',encoding='gb18030')

# look at the data

In [4]:
data.head(10)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...
5,89612,张怡,中国证券报?中证网,受到A股被纳入MSCI指数的利好消息刺激，A股市场从周三开始再度上演龙马行情，周四上午金...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",金融股一枝独秀 配置价值犹存,http://www.cs.com.cn/gppd/201706/t20170623_533...
6,89611,NaN,威锋网@http://www.feng.com/,虽然至今夏普智能手机在市场上无法排得上号，已经完全没落，并于 2013 年退出中国市场，但是...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""21""...",配骁龙660 全面屏鼻祖夏普新机酝酿中,http://www.cnbeta.com/articles/tech/623603.htm
7,89610,申玉彬 整理,中国证券报?中证网,沙漠雄鹰：震荡有利消化套牢筹码\r\n 周四开盘上证50在银行券商大蓝筹带动下一度涨近...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",博友早评：震荡有利消化套牢筹码,http://www.cs.com.cn/gppd/201706/t20170623_533...
8,89609,李杭_BJS4645,荆楚网-楚天都市报,（原标题：武汉警方一下子抓了808人，还都是俊男靓女！原来他们每天偷偷摸摸干这事！）\r\n...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""1600""...",武汉千余警察出动 抓获808名俊男靓女全是诈骗犯,http://news.163.com/17/0614/14/CMT9N8G80001899...
9,89608,吴瞬,中国证券报?中证网,6月21日，A股纳入MSCI指数尘埃落定，但当天被寄予厚望的券商股并未扛起反弹大旗。22...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",纳入MSCI指数 A股长期配置价值提升,http://www.cs.com.cn/gppd/201706/t20170623_533...


In [5]:
data.loc[0,['feature']]

feature    {"type":"科技","site":"cnbeta","commentNum":"37"...
Name: 0, dtype: object

In [6]:
def clock(func):
    def clocked(*args, **kwargs):
        t0 = timeit.default_timer()
        result = func(*args, **kwargs)
        elapsed = timeit.default_timer() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(elapsed,'s')
        #print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [7]:
def typeof(variate):
    type=None
    if isinstance(variate,int):
        type = "int"
    elif isinstance(variate,str):
        type = "str"
    elif isinstance(variate,float):
        type = "float"
    elif isinstance(variate,list):
        type = "list"
    elif isinstance(variate,tuple):
        type = "tuple"
    elif isinstance(variate,dict):
        type = "dict"
    elif isinstance(variate,set):
        type = "set"
    print(type)
    return type

# INPUT: Which is more useful, more efficient ?

In [21]:
@clock
def get_size(data, attributes):
    size = 0
    data = data[attributes]
    for i in range(len(data)):
        if isinstance(data[i],str):
            size = size + len(data[i])
    return size, size/len(data)

attributes = ['content','feature', 'title','author','url']
#attr_choosen = ['content','feature', 'title','author','url']
attr_choosen = ['content']
show = [print( i, get_size(data,i)) for i in attributes]

2.559933386999546 s
content (38729226, 432.1927665130397)
2.6356201079997845 s
feature (53450994, 596.4780439901351)
2.5343696729996736 s
title (2123231, 23.69386570845097)
2.268567680999695 s
author (505494, 5.6409815759225985)
2.3123578319991793 s
url (7472901, 83.39267500641662)


# INPUT: How do we handle the imbalance?

In [22]:
data_sampled = data.sample(n=3000)
data_pos = data_sampled[data_sampled['source'].str.contains('新华')]
print(len(data_pos))
print(len(data_pos)/len(data_sampled))

2639
0.8796666666666667


# 样本均衡
## 直接的选择数据，使其满足需求
## 分类器的weight设置
## produce the examples u need

In [10]:
data['source'] = data['source'].fillna('')
data_pos = data[data['source'].str.contains('新华')]
data_neg = data[data['source'].str.contains('新华') == 0]

#n = 5000
N_sample = 6000
data_neg = data_neg.sample(n = (int)(N_sample/2))
data_pos = data_pos.sample(n = (int)(N_sample/2))

data_sampled = data_pos.append(data_neg)
print(len(data_sampled))


# N, 3:1, 5:2, 
#5000,1:1
#20000 1:1
#30000 2:1
#30000
#print(len(data_pos))
#print(len(data) - len(data_pos))

6000


# 预处理，去标点，加label

In [11]:
def cut(string):
    return ' '.join(jieba.cut(re.sub(r'[^\w\s]',' ',string.strip())))
def preprocess(data):
    for attr in attributes:
        data[attr] = data[attr].fillna('').apply(cut)
    data['is_xinhua'] = np.where(data['source'].str.contains('新华'),1,0)
    #print(data)
    x_inputs = data.loc[:,attr_choosen]
    #print(x_inputs)
    y_inputs = data['is_xinhua'].values
    #print(y_inputs)
    return x_inputs,y_inputs
x_inputs, y_inputs = preprocess(data_sampled)       
#print(data_processed)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.930 seconds.
Prefix dict has been built succesfully.


# max_features
# 'today is a good day'-->[1,3,2,4,5]

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

#x_inputs = data_processed.loc[:,attr_choosen]
#print(inputs)
#y_inputs = data_processed['is_xinhua'].values
#print(labels)
@clock
def vectorize(x_inputs, attr_choosen):
    vectorizer = TfidfVectorizer(max_features=5000, token_pattern=r"(?u)\b\w+\b", max_df = 1.0,
                                 stop_words = None,vocabulary = None )# (?u)进入re.u，unicode匹配模式，https://stackoverflow.com/questions/35043085/what-does-u-do-in-a-regex
    vec = None
    for attr in attr_choosen:
        vec_fea = vectorizer.fit_transform(x_inputs[attr].values)
        print("vocabulary\n",vectorizer.vocabulary_, len(vectorizer.vocabulary_))
        print("stop_words\n",vectorizer.stop_words_, len(vectorizer.stop_words_))
        #typeof(vec_fea)
        #np_vec = np.array(vec_fea)
        np_vec = vec_fea.toarray()
        #print(np_vec)
        if vec is None:
            vec = np_vec
        else:
            vec = np.hstack((vec,np_vec))
        #print(attr, vec_fea.shape, np_vec.shape)
    return vec

x_vecs = vectorize(x_inputs, attr_choosen)
print(x_vecs.shape)

vocabulary
 {'公开': 934, '金额': 4707, '文物': 2797, '四个': 1564, '充满': 866, '民营': 3209, '净值': 1020, '旧': 2871, '往往': 2333, '评估': 4286, '靠': 4840, '违法行为': 4582, '1500': 23, '能够': 4023, '群众': 3962, '安装': 1948, '民生': 3208, '试验': 4300, '绿': 3921, '城': 1684, '石墨': 3665, '震荡': 4828, '总数': 2393, '研制': 3669, '房地产': 2497, '深圳市': 3339, '仅为': 652, '突破': 3760, '研发': 3670, '沿海': 3258, '创意': 1098, '肯定': 4013, '物资': 3437, '行政': 4165, '南安普敦': 1269, '认知': 4248, '2016': 46, '冠军': 1008, '于是': 568, '河南': 3252, '指数': 2626, '62': 107, '专业化': 364, '盲人': 3604, '据统计': 2646, '科研人员': 3719, '色彩': 4076, '到': 1115, '引入': 2280, '广大': 2227, '属于': 2104, '全体': 896, '影院': 2330, '青岛': 4831, '韩国': 4850, '吴': 1503, '不如': 337, '出现': 1054, '一年': 211, '节': 4079, '现状': 3482, '景观': 2924, '两大': 406, '十条': 1236, '夏普': 1743, '申报': 3530, '摆脱': 2709, '有利于': 2975, '频频': 4875, '意向': 2430, '年龄': 2213, '中期': 454, '形态': 2323, '同学': 1467, '购房': 4367, '难': 4809, '图片': 1633, '首席': 4906, '回归': 1580, '身份': 4467, '温暖': 3353, '减产': 1026, '服务中心': 299

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(
    x_vecs , y_inputs, train_size = 0.8, test_size=0.2)

# Evaluation metrics

In [14]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
@clock
def get_performance(clf, x_, y_):
    y_hat = clf.predict(x_)
    print('f1_score is: {}'.format(f1_score(y_, y_hat)))
    print('accuracy is: {}'.format(accuracy_score(y_, y_hat)))
    print('percision is: {}'.format(precision_score(y_, y_hat)))
    print('recall is: {}'.format(recall_score(y_, y_hat)))
    #print('roc_auc is: {}'.format(roc_auc_score(y_, y_hat)))
    #print('confusion matrix: \n{}'.format(confusion_matrix(y_, y_hat, labels=[0, 1])))


 

In [15]:
from sklearn.neighbors import KNeighborsClassifier
@clock
def KNN(x_train, y_train):
    clf = KNeighborsClassifier(n_neighbors = 5, weights = 'uniform', algorithm = 'auto')
    clf.fit(x_train, y_train)
    return clf
#clf.fit(X_test,y_test)
get_performance(KNN(x_train,y_train),x_test,y_test)

1.1603996230005578 s
f1_score is: 0.6192090395480226
accuracy is: 0.7191666666666666
percision is: 1.0
recall is: 0.44844517184942717
50.9136662359997 s


In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
@clock
def KNN(x_train, y_train):
    clf = KNeighborsClassifier()
    param_grid = { 'n_neighbors' : [i for i in range(1,3)]}
    grid = GridSearchCV(clf, param_grid, cv = 2, scoring='accuracy',n_jobs = -1)
    clf = grid.fit(x_train, y_train)
    clf.fit(x_train, y_train)
    print(clf.best_params_)
    return clf
#clf.fit(X_test,y_test)
get_performance(KNN(x_train,y_train),x_test,y_test)

{'n_neighbors': 1}
197.6106646870012 s
f1_score is: 0.7561697926949654
accuracy is: 0.7941666666666667
percision is: 0.9527363184079602
recall is: 0.6268412438625205
42.229468202000135 s
